In [6]:
""" Gerando o que queremos e nada mais """

# Imports
import pandas as pd

# Carregar o dataframe de transações
df = pd.read_parquet('../Data/hackathon_2025_templates/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')

# Seleciona Apenas: Colunas de pdv_id, product_id, quantity e transaction_date
df = df[['internal_product_id', 'internal_store_id', 'quantity', 'transaction_date']]

# Garante que transaction_date está em formato datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Seleciona apenas as linhas com transação_date entre 2022-01-01 e 2022-01-28
df = df[(df['transaction_date'] >= '2022-01-01') & (df['transaction_date'] < '2022-01-29')]

# Agrupa por pdv_id, product_id e transaction_date, somando quantity
df = df.groupby(['internal_store_id', 'internal_product_id', 'transaction_date'], as_index=False)['quantity'].sum()

# Cria coluna de semana (número da semana do ano)
df['week'] = df['transaction_date'].dt.isocalendar().week

# Agrupa por semana, pdv_id e product_id, somando quantity
df_weekly = df.groupby(['week', 'internal_store_id', 'internal_product_id'], as_index=False)['quantity'].sum()

# Renomeia colunas para o formato desejado
df_weekly = df_weekly.rename(columns={
    'week': 'semana',
    'internal_store_id': 'pdv',
    'internal_product_id': 'produto',
    'quantity': 'quantidade'
})

# Converte quantidade para inteiro
df_weekly['quantidade'] = df_weekly['quantidade'].astype(int)

# Salva resultado em CSV no formato (semana;pdv;produto;quantidade)
df_weekly.to_csv('resultado.csv', sep=';', encoding='utf-8', index=False)

# Exibe as primeiras linhas do resultado
df_weekly.head()

,semana,pdv,produto,quantidade
0,1,1001371918471115422,1029370090212151375,2
1,1,1001371918471115422,1120490062981954254,18
2,1,1001371918471115422,2239307647969388381,2
3,1,1001371918471115422,4353552881410365573,7
4,1,1001371918471115422,4797439216678436447,1
